<a href="https://colab.research.google.com/github/anubhav48/Catboost-vs-Lightboost-vs-XGBoost-vs-Snapboost/blob/master/Image_Data_Boosting_RSCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xgboost --quiet
!pip install catboost --quiet
!pip install lightgbm --quiet
!pip install hyperopt --quiet

     |████████████████████████████████| 64.8MB 67kB/s 


In [ ]:
from keras.datasets import cifar10
from sklearn import metrics
import catboost as ctb
from sklearn.model_selection import cross_val_score    
import warnings
warnings.filterwarnings('ignore')
import time
start_time = time.time()
import numpy as np
import pandas as pd
#import sys
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
import lightgbm as lgb
import matplotlib as mpl
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score
from catboost import Pool,CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


In [ ]:
import keras
# from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
import pickle
import sys
import os

#
# TAKEN FROM https://keras.io/examples/cifar10_cnn/
# but with load_data() pulled out to work with your own cifar data set
#

def load_batch(fpath, label_key='labels'):
    """Internal utility for parsing CIFAR data.
    # Arguments
        fpath: path the file to parse.
        label_key: key for label data in the retrieve
            dictionary.
    # Returns
        A tuple `(data, labels)`.
    """
    with open(fpath, 'rb') as f:
        if sys.version_info < (3,):
            d = pickle.load(f)
        else:
            d = pickle.load(f, encoding='bytes')
            # decode utf8
            d_decoded = {}
            for k, v in d.items():
                    d_decoded[k.decode('utf8')] = v
            d = d_decoded
      
    data = d['data']
    labels = d[label_key]
#     new_data  = np.array([])
#     new_labels = np.array([])
#     for i in range(len(labels)):
#         if labels[i] in uni:
#             np.append(new_data,data[i])
#             np.append(new_data,labels[i])
    data = data.reshape(data.shape[0], 3, 32, 32)
    return data, labels


def load_data():
    """Loads CIFAR10 dataset.
    # Returns
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """
    path = '../input/cifar10-python/cifar-10-batches-py/'

    num_train_samples = 5000

    x_train_local = np.empty((num_train_samples, 3, 32, 32), dtype='uint8')
    y_train_local = np.empty((num_train_samples,), dtype='uint8')

    for i in range(1,1):
        fpath = os.path.join(path, 'data_batch_' + str(i))
        (x_train_local[(i - 1) * 1000: i * 1000, :, :, :],
         y_train_local[(i - 1) * 1000: i * 1000]) = load_batch(fpath)
    fpath = os.path.join(path, 'test_batch')
    x_test_local, y_test_local = load_batch(fpath)
    x_test_local = x_test_local[:1000]
    y_test_local = y_test_local[:1000]
    y_train_local = np.reshape(y_train_local, (len(y_train_local), 1))
    y_test_local = np.reshape(y_test_local, (len(y_test_local), 1))

    if K.image_data_format() == 'channels_last':
        x_train_local = x_train_local.transpose(0, 2, 3, 1)
        x_test_local = x_test_local.transpose(0, 2, 3, 1)

    return (x_train_local, y_train_local), (x_test_local, y_test_local)


batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [ ]:
# data: shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [ ]:
X_train = X_train[:6000]
y_train = y_train[:6000]
X_test = X_test[:100]
y_test = y_test[:100]

In [ ]:
X_train.shape

(6000, 32, 32, 3)

In [ ]:
X_test.shape

(100, 32, 32, 3)

In [ ]:
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [4],
       [8]], dtype=uint8)

In [ ]:
 # X_train is 50000 rows of 3x32x32 values --> reshaped in 50000 x 3072
RESHAPED = 3072
X_train = X_train.reshape(6000, RESHAPED)
X_test = X_test.reshape(100, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

y_train = y_train.flatten()
y_test = y_test.flatten()

In [ ]:
X_train.shape

(6000, 3072)

In [ ]:
  # normalize the datasets
X_train /= 255.
X_test /= 255.

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

6000 train samples
100 test samples


In [ ]:
from sklearn import preprocessing
def multiclass_roc_auc_score(y_test, pred, average="macro"):
    lb = preprocessing.LabelBinarizer() 
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    pred = lb.transform(pred)
    return roc_auc_score(y_test, pred, average=average)

In [ ]:
n_estimators = [100,200,300,500]
max_features = ['auto', 'sqrt']
max_depth = [2,3,7,5,9]
min_samples_split = [0.2,0.3,0.01]
min_samples_leaf = [0.1,0.01,0.05,0.5,1, 2, 4,5,6]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf = XGBClassifier(verbose = True,gpu_id = 0,tree_method = 'gpu_hist')
xgb = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 1,\
                               cv = 2, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
model = xgb
t1=datetime.now()
model.fit(X_train,y_train)
t2=datetime.now()
execution_time_xgb = t2-t1
t3 = datetime.now()
y_pred = model.predict(X_test)
t4 = datetime.now()
predictions = [round(value) for value in y_pred]
accuracy_xgb = round(accuracy_score(y_pred,y_test),5)
print("Accuracy score = ",accuracy_xgb*100,"%")
print("Execution time = ",execution_time_xgb)
print("Prediction time = ",t4-t3)
xgb_train_time = t2-t1

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  2.5min finished


Accuracy score =  50.0 %
Execution time =  0:04:07.713843
Prediction time =  0:00:00.155914


In [ ]:
cb_hps = {  
            'depth':              [2,3,4,5,6],
            'l2_leaf_reg':        [2,3,5,10],
            'random_seed':        [5,8,10],
            'colsample_bylevel': [0.3,0.5,0.6],
            #'n_estimators':      [100,200]
        }
cb = CatBoostClassifier(iterations=5, learning_rate=0.01,loss_function='MultiClass')
cbt = RandomizedSearchCV(estimator = cb, param_distributions = cb_hps, n_iter = 1,\
                               cv = 2, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
from sklearn.metrics import roc_auc_score
model = cbt
t1=datetime.now()
model.fit(X_train,y_train)
t2=datetime.now()
execution_time_cat_boost = t2-t1
t3 = datetime.now()
pred = model.predict(X_test)
#pred = model.predict_proba(X_test)[:,1]
t4 = datetime.now()
accuracy_cat_boost= multiclass_roc_auc_score(y_test, pred, average="macro")
#accuracy_cat_boost = round(roc_auc_score(y_test,pred,multi_class="ovr",average=None),5)
#roc_auc_score(y_score=np_pred, y_true=np_label, multi_class="ovr",average=None)
print("Accuracy score = ",accuracy_cat_boost*100,"%")
print("Execution time = ",execution_time_cat_boost)
print("Prediction time = ",t4-t3)
cat_boost_train_time = t2-t1

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   30.7s finished


0:	learn: 2.2983972	total: 2.79s	remaining: 11.2s
1:	learn: 2.2939969	total: 5.61s	remaining: 8.41s
2:	learn: 2.2895067	total: 8.38s	remaining: 5.58s
3:	learn: 2.2856626	total: 11.2s	remaining: 2.81s
4:	learn: 2.2814812	total: 14s	remaining: 0us
Accuracy score =  59.19525158839003 %
Execution time =  0:00:47.712426
Prediction time =  0:00:00.022714


In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
fit_params={"early_stopping_rounds":30, 
            "eval_set" : [(X_test,y_test)],
            'eval_names': ['valid'],
            #'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
            'verbose': 100,
            'categorical_feature': 'auto'}
param_test ={'num_leaves': sp_randint(5, 10), 
             'min_child_samples': sp_randint(100, 150), 
             'min_child_weight': [1e-3, 1e-2, 1e-1, 1],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2],
             'reg_lambda': [0, 1e-1, 1, 5]}
             #This parameter defines the number of HP points to be tested
n_HP_points_to_test = 1

In [ ]:
clf = lgb.LGBMClassifier(objective = 'binary',max_depth=-1, random_state=42,
                         silent=True, n_jobs=-1, n_estimators=50)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    #scoring='binary_logloss',
    cv=2,
    refit=True,
    random_state=42,
    n_jobs = -1,
    verbose=True)

In [ ]:
train_data=lgb.Dataset(X_train,label=y_train)

num_round=50
t1=datetime.now()
gs.fit(X_train, y_train,**fit_params)
t2=datetime.now()
execution_time_lgbm = t2-t1
t3 = datetime.now()
ypred2=gs.predict(X_test)
t4 = datetime.now()

for i in range(0,len(ypred2)): 
    if ypred2[i] >=0.5:
        ypred2[i] = 1
    else:
        ypred2[i] = 0
accuracy_lgbm = round(accuracy_score(ypred2,y_test),5)
print("Accuracy score = ",accuracy_lgbm*100,"%")
print("Execution time = ",execution_time_lgbm)
print("Prediction time = ",t4-t3)
lgbm_train_time = t2-t1

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  1.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  1.2min finished


Training until validation scores don't improve for 30 rounds.
Did not meet early stopping. Best iteration is:
[50]	valid's multi_logloss: 1.67351
Accuracy score =  12.0 %
Execution time =  0:02:20.750169
Prediction time =  0:00:00.005304


In [ ]:
gs.cv_results_

{'mean_fit_time': array([72.4467504]),
 'mean_score_time': array([0.1873666]),
 'mean_test_score': array([0.3905]),
 'param_colsample_bytree': masked_array(data=[0.6247240713084175],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_min_child_samples': masked_array(data=[128],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_min_child_weight': masked_array(data=[0.1],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_num_leaves': masked_array(data=[7],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_reg_alpha': masked_array(data=[2],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_reg_lambda': masked_array(data=[0],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_subsample': masked_array(data=[0.6774801263571897],
              mask=[False],
        fill_

TypeError: 'rv_frozen' object is not iterable